In [2]:
import sqlite3
import pandas as pd

In [3]:
#Loading data

data = pd.read_csv("nobel_winners.csv").drop('Unnamed: 0', axis=1)
data

,ID,Name,Year,Country,Countries,Category,Gender,Details,Description
0,1,14th Dalai Lama,1989,Tibet,Tibet,Peace,Man,born in Tibet,NaN
1,2,A. Michael Spence,2001,United States,United States,Economics,Man,NaN,NaN
2,3,Aage Bohr,1975,Denmark,Denmark,Physics,Man,NaN,NaN
3,4,Aaron Ciechanover,2004,Israel,Israel,Chemistry,Man,NaN,NaN
4,5,Aaron Klug,1982,Lithuania,Lithuania,Chemistry,Man,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1149,1150,Emmanuelle Charpentier,2020,France,NaN,Chemistry,Woman,NaN,“For the development of a method For genome ed...
1150,1151,Jennifer Doudna,2020,United States,NaN,Chemistry,Woman,NaN,“For the development of a method For genome ed...
1151,1152,Roger Penrose,2020,United Kingdom,NaN,Physics,Man,NaN,“For the discovery that black hole Formation i...
1152,1153,Reinhard Genzel,2020,Germany,NaN,Physics,Man,NaN,“For the discovery of a supermassive compact o...


In [4]:
data2 = {'ID':  ['1', '2','3','4','5'],
        'Name': ['14th Dalai Lama', 'A. Michael Spence', 'Aage Bohr','Aaron Ciechanover','Aaron Klug'],
        'Year_of_birth': ['1935', '1943','1922','1947','1926'], 'Gender': ['Man', 'Man','Man','Man','Man'],
        }
df = pd.DataFrame(data2, columns = ['ID','Name','Year_of_birth', 'Gender'])
df

,ID,Name,Year_of_birth,Gender
0,1,14th Dalai Lama,1935,Man
1,2,A. Michael Spence,1943,Man
2,3,Aage Bohr,1922,Man
3,4,Aaron Ciechanover,1947,Man
4,5,Aaron Klug,1926,Man


In [5]:
#Create database

connection = sqlite3.connect('winners.db')

In [6]:
connection.execute("PRAGMA foreign_keys = ON")

In [7]:
connection.execute('''CREATE TABLE IF NOT EXISTS winners 
(ID INTEGER PRIMARY KEY,
Name TEXT,
Year INTEGER,
Country TEXT,
Countries TEXT,
Category TEXT, 
Gender TEXT, 
Details TEXT,
Description TEXT)''')

In [8]:
connection.execute('''CREATE TABLE IF NOT EXISTS winners_info 
(ID INTEGER PRIMARY KEY,
Name TEXT,
Year_of_birth INTEGER, 
Gender INTEGER,
FOREIGN KEY (ID) REFERENCES winners (ID)
ON UPDATE CASCADE
ON DELETE CASCADE)''')

In [10]:
data.to_sql('winners', connection, if_exists='append', index = False)

In [11]:
df.to_sql('winners_info', connection, if_exists='append', index = False)

In [12]:
#CASCADE

query = ''' DELETE FROM winners_info WHERE "Name" = "Aage Bohr"'''

connection.execute(query)

connection.commit()

connection.execute('''SELECT * FROM winners''').fetchmany(3)

#We see, that deleting something from the child database doesn't affect the parent's database

[(1,
  '14th Dalai Lama',
  1989,
  'Tibet',
  'Tibet',
  'Peace',
  'Man',
  '\xa0born in Tibet',
  None),
 (2,
  'A. Michael Spence',
  2001,
  'United States',
  'United States',
  'Economics',
  'Man',
  None,
  None),
 (3, 'Aage Bohr', 1975, 'Denmark', 'Denmark', 'Physics', 'Man', None, None)]

In [13]:

query = ''' DELETE FROM winners WHERE "Name" = "Aage Bohr"'''

connection.execute(query)

connection.commit()

connection.execute('''SELECT * FROM winners_info''').fetchmany(3)

# But, if we delete some information from the parent's database it will also delete from the child database

[(1, '14th Dalai Lama', 1935, 'Man'),
 (2, 'A. Michael Spence', 1943, 'Man'),
 (4, 'Aaron Ciechanover', 1947, 'Man')]

In [14]:
#UPDATE
# Let's add some description to the first row

query = '''UPDATE winners SET Description = 'For advocating peaceful solutions based upon tolerance and mutual respect in order to preserve the historical and cultural heritage of his people' WHERE ID = 1'''

connection.execute(query)

connection.commit()

connection.execute('''SELECT * FROM winners''').fetchone()

(1,
 '14th Dalai Lama',
 1989,
 'Tibet',
 'Tibet',
 'Peace',
 'Man',
 '\xa0born in Tibet',
 'For advocating peaceful solutions based upon tolerance and mutual respect in order to preserve the historical and cultural heritage of his people')

In [16]:
#DELETE
#And delete all information about Nobel prize winners before 2000 years

query = '''DELETE FROM winners WHERE Year < 2000'''

connection.execute(query)

connection.commit()

connection.execute('''SELECT * FROM winners''').fetchmany(3)

[(2,
  'A. Michael Spence',
  2001,
  'United States',
  'United States',
  'Economics',
  'Man',
  None,
  None),
 (4,
  'Aaron Ciechanover',
  2004,
  'Israel',
  'Israel',
  'Chemistry',
  'Man',
  None,
  None),
 (9,
  'Abhijit Banerjee',
  2019,
  'India',
  'India',
  'Economics',
  'Man',
  None,
  None)]

In [17]:
connection.commit()

In [18]:
connection.close()